### Урок 2. Создание признакового пространства#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import pandas as pd
import numpy as np
import warnings
import re
import html
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

warnings.filterwarnings("ignore")

In [2]:
data = pd.read_pickle("../data/tweet_dataframe.pkl")

In [3]:
data.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."


### 1. Создаем мешок слов с помощью CountVectorizer

In [4]:
tweet_stemmed_list = [a for b in data['tweet_stemmed'].tolist() for a in b]
tweet_lemmatized_list = [a for b in data['tweet_lemmatized'].tolist() for a in b]

In [5]:
# tweet_stemmed BOW
count_vectorizer = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')
TS_bag_of_words = count_vectorizer.fit_transform(tweet_stemmed_list)
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(TS_bag_of_words.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# tweet_lemmatized BOW
count_vectorizer = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')
TL_bag_of_words = count_vectorizer.fit_transform(tweet_lemmatized_list)
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(TL_bag_of_words.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Создаем мешок слов с помощью TfidfVectorizer

In [7]:
# tweet_stemmed values
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')
TS_values = tfidf_vectorizer.fit_transform(tweet_stemmed_list)
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(TS_values.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
375967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
375968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
375969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# tweet_lemmatized values
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')
TL_values = tfidf_vectorizer.fit_transform(tweet_lemmatized_list)
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(TL_values.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
375967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
375968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
375969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3. Натренируем Word2Vec модель 

In [9]:
sentences = data['tweet_token'].to_list()

In [10]:
modelW2V = Word2Vec(sentences=sentences, size=200, window=5, min_count=2, sg = 1, 
                    hs = 0, negative = 10, workers= 32, seed = 34)

In [11]:
modelW2V.train(sentences=sentences, total_examples=len(data['tweet_token']), epochs=20)

(9159228, 11729520)

In [12]:
modelW2V.save("./modelW2V.model")

### 4. Тестируем модель Word2Vec 

In [13]:
modelW2V = Word2Vec.load("./modelW2V.model")

In [14]:
modelW2V.most_similar(positive="dinner")

[('cookout', 0.5475519895553589),
 ('bihdaydinner', 0.5469557046890259),
 ('bolognese', 0.5419978499412537),
 ('shawarma', 0.5397335886955261),
 ('spaghetti', 0.5223090648651123),
 ('tacotuesday', 0.5175224542617798),
 ('saturdate', 0.5102052092552185),
 ('hihihi', 0.5100668668746948),
 ('sizzle', 0.5088269710540771),
 ('hamburger', 0.5085713267326355)]

In [15]:
modelW2V.most_similar(positive="trump")

[('donald', 0.539967954158783),
 ('unfavorability', 0.5243961811065674),
 ('donaldtrump', 0.5188500881195068),
 ('unfit', 0.5114142894744873),
 ('impeachment', 0.5082200765609741),
 ('dumptrump', 0.5067718029022217),
 ('chopra', 0.5054301023483276),
 ('berniebros', 0.49768999218940735),
 ('melo', 0.49718570709228516),
 ('conman', 0.4967326521873474)]

### 5. Векторное представление слова 

In [16]:
modelW2V['food']

array([ 1.91108286e-01, -5.70481181e-01, -3.78904432e-01,  8.00744891e-02,
       -5.15783906e-01,  7.52853036e-01, -5.89269936e-01,  1.97913185e-01,
        3.77343714e-01, -2.14032039e-01, -5.81473351e-01,  1.19033143e-01,
       -5.63070476e-01,  1.30838901e-01,  2.80500799e-01, -4.28586602e-01,
       -3.17066729e-01, -6.00408554e-01,  5.56066781e-02, -6.86052918e-01,
        1.65801886e-02,  2.83216327e-01, -1.70470178e-01,  4.33587804e-02,
        1.46193638e-01,  2.55785704e-01, -5.83471395e-02, -9.79931676e-04,
        3.93766582e-01,  4.41230446e-01, -1.70692295e-01, -3.74204665e-01,
        4.17581618e-01,  5.34019470e-01,  9.42180604e-02,  6.82066262e-01,
       -6.53208852e-01, -3.17708790e-01,  2.20478445e-01,  6.08783364e-01,
       -1.73922420e-01, -6.83317006e-01,  5.85294813e-02,  6.70139864e-02,
       -4.46642756e-01, -1.08502299e-01, -3.24592084e-01, -4.82709885e-01,
        2.44383737e-01, -6.45131052e-01,  2.06814080e-01, -5.65609559e-02,
        3.26128751e-01,  

### 6. Вектор для каждого твита

In [17]:
def tweet_to_vec(sentence):
    size = 200
    vec = np.array([0.0] * 200)
    vec_len = 0

    for word in sentence:
        try:  # Бывают слова которых нет в словаре
            vec += modelW2V[word].reshape((1, size))[0]
            vec_len += 1
        except KeyError:
            pass

    result = vec / vec_len

    return result    

In [18]:
wordvec_df = data['tweet_token'].apply(lambda x: tweet_to_vec(x))
wordvec_df = pd.DataFrame(wordvec_df.tolist())
wordvec_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.075758,-0.013130,-0.296668,-0.162959,-0.004053,0.328222,-0.028978,-0.073652,0.172527,-0.037078,...,0.154264,0.179728,0.124615,-0.185806,-0.021185,-0.271483,-0.186937,0.047457,0.220986,0.132382
1,0.102243,-0.167233,-0.427856,-0.204881,-0.043454,0.174056,-0.093720,0.050214,0.029848,-0.071145,...,0.240360,0.089380,0.114085,-0.095640,-0.067072,-0.278346,-0.007265,0.130720,0.179156,0.215239


In [19]:
wordvec_df.shape

(49159, 200)